In [76]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [135]:
# Name of the file to clean, Place this next to the filder of images with the same name on the desktop.
# Delect the settings part of the csv file and move the rest of the items to the top.
# Press The fast forward button up top or run each of the elements one at a time.

In [136]:
# dont load all the rows at once
name = "75_OD_V_2x2_0_0000006"
df = pd.read_csv(name + ".csv")

df.rename(columns = {' Scan No.':'Scan No.', ' Depth':'Depth', ' Layer':'Layer'}, inplace = True)
df.head(10)

cols = ['Index', 'Scan No.', 'Depth', 'Layer']
df = df[cols]
df.head(10)

,Index,Scan No.,Depth,Layer
0,1,1,0.0,IRNFL
1,2,1,0.0,IRNFL
2,3,1,0.0,IRNFL
3,4,1,0.0,IRNFL
4,5,1,0.0,IRNFL
5,6,1,0.0,IRNFL
6,7,1,0.0,IRNFL
7,8,1,0.0,IRNFL
8,9,1,0.0,IRNFL
9,10,1,0.0,IRNFL


In [137]:
# filling in NAN values in this case -1 in the depth col
df['Depth'] = df['Depth'].replace([-1], 0.0)
df.head(10)

,Index,Scan No.,Depth,Layer
0,1,1,0.0,IRNFL
1,2,1,0.0,IRNFL
2,3,1,0.0,IRNFL
3,4,1,0.0,IRNFL
4,5,1,0.0,IRNFL
5,6,1,0.0,IRNFL
6,7,1,0.0,IRNFL
7,8,1,0.0,IRNFL
8,9,1,0.0,IRNFL
9,10,1,0.0,IRNFL


In [138]:
# number of non zero value for depth
#Checking Null value counts column wise (around 15 percent of the dataset)
(df['Depth'] == 0.0).sum()

228490

In [139]:
# average start and stop
df_format = df.groupby(["Scan No.", "Layer"]).agg({'Depth':"mean"}).reset_index()
df_format = df_format.pivot(index = "Scan No.", columns = "Layer", values = "Depth")
df_format

Layer,ETPR,INL,IPL,IRNFL,IS,OPL,ORNFL,RPE
Scan No.,,,,,,,,
1,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
5,63.354500,33.347991,32.562000,27.1540,60.607000,33.871500,27.468654,66.482000
...,...,...,...,...,...,...,...,...
96,100.190673,64.475533,64.370000,56.7645,94.751336,66.561825,57.159128,104.378108
97,88.696673,58.472953,57.012491,50.3680,83.474192,59.448635,50.565664,92.258000
98,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000


In [140]:
# new df with average layer insead of avg position
layer = pd.DataFrame()

#layers ()
layer["NFL/GLC"] = df_format["ORNFL"] - df_format["IRNFL"]
layer["IPL"]     = df_format["IPL"]   - df_format["ORNFL"]
layer["INL"]     = df_format["INL"]   - df_format["IPL"]
layer["OPL"]     = df_format["OPL"]   - df_format["INL"]
layer["ONL-IS"]  = df_format["IS"]    - df_format["OPL"]
layer["OS"]      = df_format["ETPR"]  - df_format["IS"]
layer["RPE"]     = df_format["RPE"]   - df_format["ETPR"]
layer["TOTAL"]   = df_format["RPE"]   - df_format["ORNFL"]
layer["OVERALL"] = (layer["NFL/GLC"] + layer["IPL"] + layer["INL"] + layer["OPL"] + layer["ONL-IS"] + layer["OS"] + layer["RPE"]) / 7
layer

,NFL/GLC,IPL,INL,OPL,ONL-IS,OS,RPE,TOTAL,OVERALL
Scan No.,,,,,,,,,
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.314654,5.093346,0.785991,0.523509,26.735500,2.747500,3.127500,39.013346,5.618286
...,...,...,...,...,...,...,...,...,...
96,0.394628,7.210872,0.105533,2.086292,28.189511,5.439336,4.187435,47.218979,6.801944
97,0.197664,6.446827,1.460463,0.975682,24.025556,5.222481,3.561327,41.692336,5.984286
98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [141]:
# all data
layer.to_csv(name + "Mouse_OUT_FULL.csv", encoding='utf-8')

# non zero rows
mask = layer["OVERALL"] != 0
layer[mask]
layer.to_csv(name + "Mouse_OUT.csv", encoding='utf-8')

In [142]:
# remaking measurment arrarys (heatmap and original)

In [143]:
#creating the image sets to be processed
def openDirectory(directory_name):
    imgs = []
    if os.path.isdir(directory_name) == True:
        for x in os.listdir(directory_name):
            if x.endswith(".TIFF"):
                imgs.append(os.path.join(directory_name, x))
    else:
        return "Input is not a valid drectory"
    return imgs

imgs = openDirectory(name)

In [144]:
df.head(10)

,Index,Scan No.,Depth,Layer
0,1,1,0.0,IRNFL
1,2,1,0.0,IRNFL
2,3,1,0.0,IRNFL
3,4,1,0.0,IRNFL
4,5,1,0.0,IRNFL
5,6,1,0.0,IRNFL
6,7,1,0.0,IRNFL
7,8,1,0.0,IRNFL
8,9,1,0.0,IRNFL
9,10,1,0.0,IRNFL


In [145]:
ETPR  = np.zeros((100,1000), dtype = "int32")
INL   = np.zeros((100,1000), dtype = "int32")
IPL   = np.zeros((100,1000), dtype = "int32")
IRNFL = np.zeros((100,1000), dtype = "int32")
IS    = np.zeros((100,1000), dtype = "int32")
OPL   = np.zeros((100,1000), dtype = "int32")
ORNFL = np.zeros((100,1000), dtype = "int32")
RPE   = np.zeros((100,1000), dtype = "int32")

mask = df["Depth"] != 0
df = df[mask]

def formats(row):
    scale = row["Index"] - 1
    while scale > 1000:
        scale = scale - 1000
        
    if row["Layer"] == "ETPR":
        ETPR[row["Scan No."] - 1][scale] = int(row["Depth"])
    elif row["Layer"] == "INL":
        INL[row["Scan No."] - 1][scale] = int(row["Depth"])
    elif row["Layer"] == "IPL":
        IPL[row["Scan No."] - 1][scale] = int(row["Depth"])
    elif row["Layer"] == "IRNFL":
        IRNFL[row["Scan No."] - 1][scale] = int(row["Depth"])  
    elif row["Layer"] == "IS":
        IS[row["Scan No."] - 1][scale] = int(row["Depth"])
    elif row["Layer"] == "OPL":
        OPL[row["Scan No."] - 1][scale] = int(row["Depth"]) 
    elif row["Layer"] == "ORNFL":
        ORNFL[row["Scan No."] - 1][scale] = int(row["Depth"])
    elif row["Layer"] == "RPE":
        RPE[row["Scan No."] - 1][scale] = int(row["Depth"]) 
    return row

df.apply(formats, axis = 1)

,Index,Scan No.,Depth,Layer
32375,32376,5,112.6729,IRNFL
32376,32377,5,112.3271,IRNFL
32377,32378,5,112.0000,IRNFL
32378,32379,5,111.6729,IRNFL
32379,32380,5,111.3271,IRNFL
...,...,...,...,...
775681,768682,97,267.0000,RPE
775682,768683,97,267.0000,RPE
775683,768684,97,267.0000,RPE
775684,768685,97,267.0000,RPE


In [146]:
ETPR[10]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [147]:
# Load each image for point entry and resaving each image
import cv2
import matplotlib.image as mpimg

save_location = os.path.join(os.getcwd(), "OutImages")

# making diretories
if os.path.isdir(save_location) == False:
    os.mkdir(save_location)
if os.path.isdir(os.path.join(save_location, name)) == False:
    os.mkdir(os.path.join(save_location, name))

#each layer its colour
for x in range(0, len(imgs)):
    image = cv2.imread(imgs[x])
    for y in range(0, 1000):
        if   IRNFL[x][y] != 0: image[IRNFL[x][y]][y]  = [255,0,0]
        if   ORNFL[x][y] != 0: image[ORNFL[x][y]][y]  = [0,0,255]
        if   IPL[x][y]   != 0: image[IPL[x][y]][y]    = [0,255,0]
        if   INL[x][y]   != 0: image[INL[x][y]][y]    = [255,105,180]
        if   OPL[x][y]   != 0: image[OPL[x][y]][y]    = [255,128,0]
        if   IS[x][y]    != 0: image[IS[x][y]][y]     = [0,255,255]
        if   ETPR[x][y]  != 0: image[ETPR[x][y]][y]   = [0,0, 255]
        if   RPE[x][y]   != 0: image[RPE[x][y]][y]    = [255,255,0]
    mpimg.imsave(os.path.join(save_location, imgs[x]), image)

In [148]:
# heatmap intro - getting the heatmap total value matrix working

total = np.zeros((100, 1000), dtype = "int32")
for x in range(0, 100):
    for y in range(0,1000):
        total[x][y] = int(RPE[x][y]) - int(ORNFL[x][y])

new = []
for x in range(0, 100):
    for y in range(0, 10):
        new.append(total[x])

total = np.array(new)

In [149]:
for x in range(0, 1000):
    for y in range(0, 1000):
        m = 100
        if total[x][y] - m < 0:
            total[x][y] = 0
        else: 
            total[x][y] = total[x][y] - m

blank_image = np.zeros((1030,1000,3), np.uint8)
color_gradient = [[0,100,250], [0,95,235], [0,90,255], [0,85,212], [0,80,199], [0,75,186],
                        [0,70,175],  [0,65,162], [0,60,151], [0,55,138], [0,50,125],                       #Blue

                        [9,255,0], [8,245,0], [8,235,0], [7,225,0], [7,215,0], [6,205,0],
                        [6,195,0], [5,185,0], [4,175,0], [4,165,0], [3,155,0],                               #Green

                        [255,255,0], [250,250,0], [245,245,0], [240,240,0], [235,235,0],
                        [230,230,0], [225,225,0], [220,220,0], [215,215,0], [210,210,0], [205,205,0],        #Yellow

                        [255,119,0], [245,117,0], [235,110,0], [225,105,0], [215,100,0], [205,96,0],
                        [195,91,0], [185,86,0], [175,83,0], [165,78,0], [155,72,0],                          #Orange

                        [255,0,0], [240,0,0], [225,0,0], [210,0,0], [195,0,0], [180,0,0],
                        [165,0,0], [150,0,0], [135,0,0], [120,0,0], [105,0,0]]                               #Red

for x in range(0,1000):
    for y in range(0,1000):
        if total[x][y] == 0:
             blank_image[x][y] = [0,0,0]
        elif total[x][y] >= len(color_gradient):
            blank_image[x][y] = [105,0,0]
        else:
            blank_image[x][y] = color_gradient[total[x][y]]
            
# blank lines


# the colour key at the bottom of the image
color_key = [x for x in color_gradient for i in range(10)]
while len(color_key) < 1000:
    color_key.insert(0, [0,0,0])
    if len(color_key) < 1000:
        color_key.append([0,0,0])
        
for x in range(1020, 1030):
    for y in range(0,len(color_key)):
        blank_image[x][y] = color_key[y]

maxval = str(np.max(total))
font = cv2.FONT_HERSHEY_SIMPLEX

cv2.putText(blank_image, "Min" ,(230, 1010), font, 1,(255,255,255),1,cv2.LINE_AA)
cv2.putText(blank_image, str(m),(300, 1010), font, 1,(255,255,255),1,cv2.LINE_AA)

cv2.putText(blank_image, "Max" ,(660, 1010), font, 1,(255,255,255),1,cv2.LINE_AA)
cv2.putText(blank_image, maxval,(730, 1010), font, 1,(255,255,255),1,cv2.LINE_AA)
mpimg.imsave(save_location + str(os.sep) + str(m) + "Heatmap" + ".TIFF", blank_image)